In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import gurobipy as gp
import math
from gurobipy import *
GRB_LICENSE_FILE="/Users/juanamejia/gurobi.lic"

|Estudiante|Código|Correo|
|-----|-----|----|
|Juana Mejía Botero|20221512|j.mejia17|
|Daniela Ricaurte Echeverry|201822966|d.ricaurte|

# Proyecto - Entrega 3

# Table de Contenido
[[_TOC_]]

## Modelo Matemático

### Variables del Problema
- $y_{ij}$ variable binaria que representa el *estado de la llave de interconexión* entre i y j, donde esta será 1 si la llave está abierta y 0 si se encuentra cerrada.  
- $P_{ij}$ variable continua que representa el flujo de *potencia activa* entre los nodos $i$ y $j$.
- $Q_{ij}$ variable continua que representa el flujo de *potencia reactiva* entre los nodos $i$ y $j$.
- $I_{ij}$ variable continua que representa el flujo de corriente entre los nodos $i$ y $j$.
- $V_{i}$ variable continua  representa el voltaje en el nodo $i$.


### Parámetros del Problema

- $P_{i}^{D}$ representa la demanda de potencia activa en el nodo $i$ 

- $Q_{i}^{D}$  representa la demanda de potencia  reactiva en el nodo $i$.

- $R_{ij}$ representa la resistencia de la linea entre los nodos $i$ y $j$ 

- $X_{ij}$ representa la reactancia de la linea entre los nodos $i$ y $j$.

In [ ]:
# Base de datos
file = open("/Users/juanamejia/Desktop/uni/SVII/FR/datos14.txt", "r",encoding='latin-1')

print(file.read())
print()



In [ ]:
import re

# Define a dictionary to store the values
values = {}
branches = []
bus_demand = []


# Define regular expressions to match the variables and their values
patterns = {
    'nref': r'nref\s*=\s*([\d.]+);',
    'vref': r'vref\s*=\s*([\d.]+);',
    'vbase': r'vbase\s*=\s*([\d.]+);',
    'sbase': r'sbase\s*=\s*([\d.]+);',
    'tol': r'tol\s*=\s*([\d.eE+-]+);',  # Updated for exponent notation
    'vmin': r'vmin\s*=\s*([\d.]+);',
    'vmax': r'vmax\s*=\s*([\d.]+);',
    'zbase': r'zbase\s*=\s*([\d.]+);',
}

branch_pattern = r'ramos\s*=\s*\[(.*?)\];'
bus_demand_pattern = r'barras\s*=\s*\[(.*?)\];'

# Read the content of the file
file_path = "/Users/juanamejia/Desktop/uni/SVII/FR/datos14.txt"
with open(file_path, "r",encoding='latin-1' ) as file:
    text = file.read()

# Extract the values using regular expressions
for key, pattern in patterns.items():
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        if key == 'tol':
            print(key)
            base, exponent = match.group(1).split('e')
            values[key] = float(base) * 10 ** int(exponent)
        else:
            values[key] = float(match.group(1))
print (values)
# Print the extracted values
for key, value in values.items():
    print("items")
    print(f"{key}: {value}")
    

# Extract branch data using the pattern
match = re.search(branch_pattern, text, re.DOTALL)
if match:
    branch_data = match.group(1)

    # Split the data into individual lines and remove comments
    branch_lines = branch_data.split('\n')
    branch_lines = [line.strip() for line in branch_lines if not line.strip().startswith('%')]

    # Split each line into individual values and convert them to floats
    for line in branch_lines:
        valores = line.split()
        if len(valores) == 4:
            branch = [float(valor) for valor in valores]
            branches.append(branch)

# Print the extracted branch data
for branch in branches:
    print(branch)

match = re.search(bus_demand_pattern, text, re.DOTALL)
if match:
    bus_demand_data = match.group(1)

    # Split the data into individual lines and remove comments
    bus_demand_lines = bus_demand_data.split('\n')
    bus_demand_lines = [line.strip() for line in bus_demand_lines if not line.strip().startswith('%')]

    # Split each line into individual values and convert them to floats
    for line in bus_demand_lines:
        valores = line.split()
        if len(valores) >= 4:
            bus = [int(valores[0]), float(valores[1]), float(valores[2]), float(valores[3])]
            bus_demand.append(bus)

# Print the extracted bus demand data
for bus in bus_demand:
    print(bus)



In [ ]:

nref=values.get("nref")
nodes = list(range(1,int(nref)+1))
print(nodes)

In [ ]:
potencias = ['Activa', 'Reactiva']
pd = {}
qd = {}

for bus in bus_demand:
    node = bus[0]
    active_power = bus[1]
    reactive_power = bus[2]
    pd[(node)] = active_power
    qd[( node)] = reactive_power

print("active")
for key, value in pd.items():

    print(key, value)

print("reactive")
for key, value in qd.items():
    print(key, value)

In [ ]:
pd.get(14)

In [ ]:
impedencia = ["Resistencia", "Reactancia" ]

#resistencia
r = {}
#Reactancia
x = {}
#arcos
B = []

for branch in branches:
    from_node = int(branch[0])
    to_node = int(branch[1])
    resistance = branch[2]
    reactance = branch[3]
    
    
    B.append( (from_node, to_node))
    r[( from_node, to_node)] = resistance*0.01
    x[( from_node, to_node)] = reactance*0.01

print("Resistencia")
for key, value in r.items():
    print(key, value)

print("Reactancia")
for key, value in x.items():
    print(key, value)
    
print("arcos")
print(B)

In [ ]:
print(r.get((14,13)))

In [ ]:
# Crear un grafo dirigido
G = nx.DiGraph()

# Añadir nodos
for node in nodes:
    G.add_node(node)
    

    

Creación del modelo


Para le modelo t=1

In [ ]:
# Create optimization model
m = gp.Model("RSD")
#Constantes

#tolerancia
tol=0.0000001

# b=lo que genera/consume cada nodo
b_v= 0.1

# tension minimima kV
vmin=float(values.get("vmin"))*float(values.get("vbase"))

# tension maxima kV
vmax= float(values.get("vmax"))*float(values.get("vbase"))

# Potencia base kW
sbase= values.get("sbase")/0.8

print(vmin, vmax)
# Variables
# Estado de la llave del arco i y j
y =  m.addVars(nodes, nodes,  vtype=GRB.BINARY, name="y")

# Flujo de potencia activa entre los nodos i y j 
P =  m.addVars(nodes,nodes, vtype=GRB.CONTINUOUS, name="P")

# Flujo de potencia reactiva entre los nodos i y j 
Q = m.addVars(nodes,nodes,  vtype=GRB.CONTINUOUS, name="Q")

# Flujo de Corriente entre los nodos i y j
I = m.addVars(nodes,nodes,  vtype=GRB.CONTINUOUS, name="I")

# Raiz cuadrada del flujo de Corriente entre los nodos i y j
Isqrt = m.addVars(nodes,nodes,  vtype=GRB.CONTINUOUS, name="Isqrt")

# Voltaje del nodo i
V = m.addVars(nodes,  vtype=GRB.CONTINUOUS, lb=vmin, ub=vmax, name="V")

# Raiz cuadrada del voltaje del nodo i
Vsqrt = m.addVars(nodes, vtype=GRB.CONTINUOUS, name="Vsqrt")

# Delta del V
deltaV= m.addVars(nodes,nodes,  vtype=GRB.CONTINUOUS, name="deltaV")


In [ ]:
print (r[14,13])

In [ ]:
for (i,j)in B:
    print(r[i, j] )

In [ ]:
for (i,j)in B:
    print(x[i, j] )

In [ ]:
for i in pd:
    print(pd[i] )

In [ ]:
for (i, j) in B:
    m.addConstr(Isqrt[i, j] * Isqrt[i, j] == I[i, j], name=f"sqrt_constraint_I_{i}_{j}")

In [ ]:
for i in nodes:
    m.addConstr(Vsqrt[i] * Vsqrt[i] == V[i], name=f"sqrt_constraint_V_{i}_{j}")

In [ ]:
m.update()

### Función Objetivo

$$
Min \sum\limits_{ij \in {B}} {{R_{ij}}I_{ij}^{sqr}};
$$


In [ ]:
FO = LinExpr()
for (i, j) in B:
    FO += Isqrt[i,j] * r[i, j]

m.setObjective(FO, GRB.MINIMIZE)
    

### Restricciones del problema

1. Equilibrio de potencia activa: esta restricción regula que la cantidad total de potencia activa que es consumida en el sistema eléctrico sea igual a la generada y entregada al sistema, además debe ser igual a la demandada del nodo. 

$$
\sum\limits_{ki \in {B}} {{P_{ki}}}   - \sum\limits_{ij \in {B}} {\left( {{P_{ij}} + {R_{ij}}I_{ij}^{sqr}} \right)} + P_{i,t}^S = P_{i,t}^D;\forall i \in {N}
$$

In [ ]:
print(values.get("sbase"))

In [ ]:
#si
for i in nodes:
    # Create a linear expression for the first part
    expr1 = quicksum(P[k, i] for (k, i) in B)
    
    # Create a linear expression for the second part
    expr2 = quicksum((P[i, j] + r[i, j] * Isqrt[i, j]) for (i, j) in B)
    
    # Subtract the second expression from the first and add sbase
    r1 = expr1 - expr2 + sbase
    
    # Add the constraint
    m.addConstr(r1 == pd[i], name=f"demanda potencia activa _{i}")


2. Equilibrio de potencia reactiva: permite que la cantidad total de potencia reactiva que es consumida en el sistema eléctrico sea igual a la generada y entregada al sistema y debe igualar la demanda del nodo.

$$
\sum\limits_{ki \in {B}} {{Q_{ki,t}}}   - \sum\limits_{ij \in {B}} {\left( {{Q_{ij,t}} + {X_{ij}}I_{ij,t}^{sqr}} \right)}  + Q_{i,t}^S= Q_{i,t}^D;\forall i \in {N},    \forall t \in {T}
$$ 

In [ ]:
#si
for i in nodes: 
    expr1= quicksum(Q[k,i] for (k,i) in B)
    expr2= quicksum((Q[i,j] + r[i,j]*Isqrt[i,j]) for (i,j) in B)
    r2= expr1-expr2+ sbase
    m.addConstr(r2== qd[i], name=f"demanda potencia reactiva _{i}")

3. Restricción del limite de la magnitud de tensión. El voltaje debe encontrarse entre un rango especifico. 

$$
{V_{i,t}^{sqr} = V_{j,t}^{sqr} + 2\left( {{R_{ij}}{P_{ij,t}} + {X_{ij}}{Q_{ij,t}}} \right)}- Z_{ij}^2{I_{ij,t}^{sqr}} + \Delta_{ij,t}^{V}; \forall ij \in {B},  \forall  t \in {T}
$$

$$
{V_{i}^{sqr} = V_{j}^{sqr} + 2\left( {{R_{ij}}{P_{ij}} + {X_{ij}}{Q_{ij}}} \right)}- (x_{ij}^2+r_{i,j}^2){I_{ij}^{sqr}} + \Delta_{ij}^{V}; \forall ij \in {B}
$$

$Z_{i,j}=x_{ij}^2+r_{i,j}^2$

In [ ]:
#no pero da 0 
for i,j in B: 
    expr1= 2*(P[(i, j)] * r[i,j] +Q[(i, j)] *x[i,j])
    expr2= (x[i,j]**2 + r[i,j]**2) *Isqrt[i,j]
    r3= Vsqrt[(j)] + expr1 - expr2 + deltaV[i,j]
    
    m.addConstr(Vsqrt[i]==r3,name=f"limite de magnitud de tensión")

4. Restricción que permite regular la magnitud de tensión entre los nodos del sistema con respecto al estado de conexión/desconexión de las llaves de interconexión entre los nodos. 

$$
- {b^V}( {1 - {y_{ij}^B}}) \le \Delta _{ij}^V \le {b^V}( {1 -  {y_{ij}^B}});\forall ij \in B
$$
b= lo que genera/consume cada nodo


In [ ]:
#no
for (i,j) in B: 
    expr1= -b_v *(1 - y[i, j])
    m.addConstr(expr1 <= deltaV[(i, j)] ,name=f"regular la magnitud minima de tensión entre los nodos del sistema con respecto a las llaves")
    expr2 = b_v * (1 - y[i, j])
    m.addConstr(deltaV[(i, j)]<= expr2,name=f"regular la magnitud maxima de tensión entre los nodos del sistema con respecto a las llaves")

   

5. Restricción que permite que se sigan las leyes de la conservación de energía en el sistema eléctrico y tiene en cuenta la relación entre las magnitudes de tensión, corriente y potencias activas y reactivas
$$
V_{j}^{sqr} I_{ij}^{sqr} = P_{ij}^2 +  Q_{ij}^2;\forall ij \in {B}
$$

Esta se debe de linealizar, para lograr que sea una restricción optima de LP

In [ ]:
#no pero da 0 
for (i,j) in B:
    m.addConstr(Vsqrt[j] * Isqrt[i, j] == P[i, j] * P[i, j] + Q[i, j] * Q[i, j],
                    name=f"restriccion_cuadratica_{i}_{j}")

6. Conservación de potencia en el sistema eléctrico. 

$$
{ {\underline V }^2} \le V_{i}^{sqr} \le {\overline V }^2;\forall i \in N
$$

In [ ]:
#si
for i in nodes:
    m.addConstr(vmin**2<= Vsqrt[i], name =f" Conservación_minima_de_potencia_{i}")
    m.addConstr(Vsqrt[i]<=vmax**2, name =f" Conservación_maxima_de_potencia_{i}")

7. Límites al cuadrado de la magnitud de la corriente entre los nodos i y j cuando la llave de interconexión se encuentre abierta. 

$$
0 \le I_{ij}^{sqr} \le \overline I_{ij}^2 y_{ij}^B;\forall ij \in {B}
$$

$  \overline I_{ij} = V_{max} / R_{i,j}$


In [ ]:
#no pero da 0 
for (i,j) in B:
    m.addConstr(0<=Isqrt[i,j], name=f"magnitud_minima_Corriente_{i}_{j}")
    m.addConstr(Isqrt[i,j]<= (vmax/r[i,j])**2*y[i,j], name=f"magnitud_maxima_Corriente_{i}_{j}")
    

8. Restricción de la variable de estado de funcionamiento de los interruptores de interconexión $y_{ij,t}^B$.

$$
y_{ij,t}^B \in \left\{ {0,1} \right\};\forall ij \in {B}, \forall t \in {T}
$$

9. Restricción de radialidad

$$\sum_{ij \in B}{ y_{ij,t}^B} = |N|- |N_{sub}|$$

Para le caso Nsub= 1, entonces

$$\sum_{ij \in B}{ y_{ij,t}^B} = |N|- 1$$


In [ ]:
print(nodes)
print(len(nodes))

In [ ]:
valor=0

for (i,j) in B:
    valor+=y[i,j]
    
m.addConstr(valor ==len(nodes)-1, name=f"Restriccione_de_radialidad")

In [ ]:
m.update()

In [ ]:
m.setParam('NonConvex', 2)
m.optimize()